In [223]:
# Allows us to use the BigQuery "magic" (%%bigquery)
%load_ext google.cloud.bigquery

# Required so python actually gets re-executed every time
%load_ext autoreload
%autoreload 2

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [224]:
import os
import sys

parts = os.path.normpath(os.getcwd()).split(os.path.sep)
relative_path_parts = parts[:parts.index('pulse-data') + 1]
abs_path = os.path.join('/', *relative_path_parts)

if sys.path[0] != abs_path:
    sys.path.insert(0, abs_path)

from datetime import datetime
from google.cloud import bigquery
from typing import Optional

from recidiviz.big_query.big_query_client import BigQueryClientImpl
from recidiviz.big_query.view_update_manager import TEMP_DATASET_DEFAULT_TABLE_EXPIRATION_MS
from recidiviz.ingest.direct.controllers.direct_ingest_view_collector import DirectIngestPreProcessedIngestViewCollector
from recidiviz.ingest.direct.views.direct_ingest_big_query_view_types import DirectIngestPreProcessedIngestView, RawTableViewType
from recidiviz.utils import regions
from recidiviz.utils.environment import GCP_PROJECT_STAGING
from recidiviz.utils.metadata import local_project_id_override, project_id
from google.cloud.bigquery import magics
magics.context.progress_bar_type = None

region_code = 'US_PA'
view_tag = 'supervision_period'
lower_bound_dt = datetime.fromisoformat('2020-06-11')
upper_bound_dt = datetime.fromisoformat('2020-09-21')
dataset_prefix = 'FILL_THIS_OUT'


In [225]:
# Get the view
with local_project_id_override(GCP_PROJECT_STAGING):
    region = regions.get_region(region_code, is_direct_ingest=True)
    collector = DirectIngestPreProcessedIngestViewCollector(region, [])

    views_by_tag = {
        builder.file_tag: builder.build()
        for builder in collector.collect_view_builders()}

    view = views_by_tag[view_tag]

In [226]:
# Create a dataset for materialized tables
validation_sandbox_dataset_id = f'{dataset_prefix}_{view_tag}_validation'

with local_project_id_override(GCP_PROJECT_STAGING):
    bq_client = BigQueryClientImpl()

    validation_dataset = bq_client.dataset_ref_for_id(validation_sandbox_dataset_id)

    print(f'Creating dataset [{validation_dataset.project}.{validation_dataset.dataset_id}] ...')
    bq_client.create_dataset_if_necessary(
        validation_dataset,
        default_table_expiration_ms=TEMP_DATASET_DEFAULT_TABLE_EXPIRATION_MS
    )
    print(f'Done creating dataset [{validation_dataset.project}.{validation_dataset.dataset_id}] ...')

Creating dataset [recidiviz-staging.ageiduschek_supervision_period_validation] ...
Done creating dataset [recidiviz-staging.ageiduschek_supervision_period_validation] ...


In [227]:
# Defines a function that can be used to materialize a table for use in later queries
def materialize_query_with_name(
    dataset_ref: bigquery.DatasetReference,
    view_tag: str,
    query_name: str,
    query: str,
    query_dt: Optional[datetime] = None
):
    table_id = f'{view_tag}_{query_name}'
    print(f'Writing {query_name} query to [{dataset_ref.project}.{dataset_ref.dataset_id}.{table_id}]...')
    
    parameters = [bigquery.ScalarQueryParameter('update_timestamp', 'DATETIME', query_dt)] if query_dt else None
    create_job = bq_client.create_table_from_query_async(
        dataset_id=dataset_ref.dataset_id,
        table_id=table_id,
        query=query,
        query_parameters=parameters,
        overwrite=True
    )
    create_job.result()
    print(f'Finished writing {query_name} query.')

In [228]:
# Query the view two times and materialize the results to tables so we can analyze query determinism 
with local_project_id_override(GCP_PROJECT_STAGING):
    latest_query = view.expanded_view_query(
        config=DirectIngestPreProcessedIngestView.QueryStructureConfig(
            raw_table_view_type=RawTableViewType.LATEST,
        )
    )
    
    materialize_query_with_name(
        dataset_ref=validation_dataset,
        view_tag=view_tag,
        query_name='latest',
        query=latest_query,
    )
    
    materialize_query_with_name(
        dataset_ref=validation_dataset,
        view_tag=view_tag,
        query_name='latest_duplicate',
        query=latest_query,
    )

    print('Load complete')

Writing latest query to [recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest]...
Finished writing latest query.
Writing latest_duplicate query to [recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest_duplicate]...
Finished writing latest_duplicate query.
Load complete


In [229]:
%%bigquery
SELECT COUNT(*)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`

,f0_
0,1203603


In [230]:
%%bigquery
SELECT COUNT(*)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest_duplicate`

,f0_
0,1203603


In [231]:
%%bigquery
# Checks that two executions of the query are deterministic (should return 0 rows)
SELECT * # EXCEPT (<add column names here to narrow down what is changing>)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`
EXCEPT DISTINCT
SELECT * # EXCEPT (<add column names here to narrow down what is changing>)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest_duplicate`


,parole_number,period_sequence_number,supervision_types,admission_reason,start_date,termination_reason,termination_date,county_of_residence,district_office,district_sub_office_id,supervision_location_org_code,supervision_level,supervising_officer_name,condition_codes


In [232]:
%%bigquery --params {"parole_number": "ENTER_PAROLE_NUMBER_TO_DEBUG_HERE"}
# This query can be used to debug what has changed between two runs of the query
SELECT
    *
FROM (
    SELECT '2NEW' AS t, *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`
    UNION ALL
    SELECT '1OLD' AS t, *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest_duplicate`
)
WHERE parole_number = @parole_number
ORDER BY t, period_sequence_number, start_date, termination_date

,t,parole_number,period_sequence_number,supervision_types,admission_reason,start_date,termination_reason,termination_date,county_of_residence,district_office,district_sub_office_id,supervision_location_org_code,supervision_level,supervising_officer_name,condition_codes


In [233]:
# Query the and materialize the view with two different date bounds so we can analyze view stability over time 
with local_project_id_override(GCP_PROJECT_STAGING):
    parameterized_query = view.expanded_view_query(
        config=DirectIngestPreProcessedIngestView.QueryStructureConfig(
            raw_table_view_type=RawTableViewType.PARAMETERIZED,
        )
    )

    materialize_query_with_name(
        dataset_ref=validation_dataset,
        view_tag=view_tag,
        query_name='lower_bound',
        query=parameterized_query,
        query_dt=lower_bound_dt
    )
    
    materialize_query_with_name(
        dataset_ref=validation_dataset,
        view_tag=view_tag,
        query_name='upper_bound',
        query=parameterized_query,
        query_dt=upper_bound_dt
    )

    print('Load complete')

Writing lower_bound query to [recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_lower_bound]...
Finished writing lower_bound query.
Writing upper_bound query to [recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_upper_bound]...
Finished writing upper_bound query.
Load complete


In [234]:
%%bigquery
SELECT COUNT(*)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_lower_bound`

,f0_
0,1151263


In [235]:
%%bigquery
SELECT COUNT(*)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_upper_bound`

,f0_
0,1173526


In [236]:
%%bigquery
# Returns rows that changed between two date bounded queries that we don't expect to change (dates are in the past)
SELECT
    COUNT(*) AS num_unexpected_changes,
    COUNT(DISTINCT parole_number) AS num_people_unexpected_changes, 
FROM (
    SELECT *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_upper_bound`
    EXCEPT DISTINCT
    SELECT *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_lower_bound`
)
WHERE start_date < '2020-01-01' AND termination_date < '2020-01-01'
# , county_of_residence, condition_codes

,num_unexpected_changes,num_people_unexpected_changes
0,17965,4288


In [237]:
%%bigquery --params {"parole_number": "ENTER_PAROLE_NUMBER_TO_DEBUG_HERE"}
# This query can be used to debug what has changed between two runs of the query
SELECT *
FROM (
    SELECT '2NEW' AS t, *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_upper_bound`
    UNION ALL
    SELECT '1OLD' AS t, *
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_lower_bound`
)
WHERE parole_number = @parole_number
ORDER BY t, period_sequence_number, start_date, termination_date


,t,parole_number,period_sequence_number,supervision_types,admission_reason,start_date,termination_reason,termination_date,county_of_residence,district_office,district_sub_office_id,supervision_location_org_code,supervision_level,supervising_officer_name,condition_codes


In [238]:
%%bigquery --params {"comparison_date": "2019-12-31", "state_code": "US_PA", "id_type": "US_PA_PBPP"}
# Does a population check for a given date and compares to external accuracy metrics
WITH 
overlapping_ips AS (
    SELECT DISTINCT person_external_id, person_id
    FROM `recidiviz-staging.state.state_incarceration_period` ip
    JOIN (
        SELECT state_code, person_id, external_id AS person_external_id
        FROM `recidiviz-staging.state.state_person_external_id` eid
        WHERE eid.id_type = @id_type
    )
    USING (state_code, person_id)
    WHERE state_code = @state_code 
        AND admission_date <= @comparison_date 
        AND (release_date IS NULL OR release_date > @comparison_date)
        AND custodial_authority != 'SUPERVISION_AUTHORITY'
        AND ip.external_id IS NOT NULL
),
my_query AS (
    SELECT parole_number, start_date, termination_date
    FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`
    WHERE 
        start_date <= @comparison_date  AND (termination_date IS NULL OR termination_date > @comparison_date ) 
        AND parole_number NOT IN (SELECT DISTINCT person_external_id FROM overlapping_ips)
),
their_query AS (
    SELECT region_code, date_of_supervision, person_external_id AS parole_number
    FROM `recidiviz-staging.validation_external_accuracy_tables.supervision_population_person_level`
    WHERE region_code = @state_code
    AND date_of_supervision = @comparison_date 
),
current_query AS (
    # Can't use dataflow metrics because some people have two parole numbers and the wrong
    # one gets spit out of the pipelines.
    #     SELECT DISTINCT person_external_id AS parole_number
    #     FROM `recidiviz-staging.dataflow_metrics_materialized.most_recent_supervision_population_metrics`
    #     WHERE state_code = @state_code AND date_of_supervision = @comparison_date
    
    SELECT DISTINCT eid.external_id AS parole_number
    FROM 
    `recidiviz-staging.state.state_supervision_period`
    JOIN
    `recidiviz-staging.state.state_person_external_id` eid
    USING (person_id)
    WHERE id_type = @id_type
    AND 
        start_date <= @comparison_date  AND (termination_date IS NULL OR termination_date > @comparison_date ) 
        AND person_id NOT IN (SELECT DISTINCT person_id FROM overlapping_ips)

)
SELECT
    COUNTIF(my_query.parole_number IS NULL) AS undercounted_people,
    COUNTIF(their_query.parole_number IS NULL) AS overcounted_people,
    COUNTIF(their_query.parole_number IS NOT NULL AND my_query.parole_number IS  NOT NULL) AS correctly_counted_people
FROM 
    my_query 
FULL OUTER JOIN
    their_query 
USING (parole_number);

,undercounted_people,overcounted_people,correctly_counted_people
0,827,541,40874


In [239]:
%%bigquery

# Counts overlapping periods
SELECT COUNT(DISTINCT period.parole_number)
FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest` period
LEFT OUTER JOIN
    `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest` other_period
ON period.parole_number = other_period.parole_number 
    AND period.period_sequence_number != other_period.period_sequence_number
    AND period.start_date <= other_period.start_date 
    AND (period.termination_date IS NULL OR period.termination_date > other_period.start_date)
    -- Ignore zero-day periods on the start_date
    AND (other_period.start_date != other_period.termination_date 
            OR period.start_date < other_period.start_date)
WHERE other_period.parole_number IS NOT NULL

,f0_
0,0


In [244]:
%%bigquery --params {"lower_bound_date": "2018-01-01"}
WITH date_counts AS (
    SELECT supervision_date, COUNT(*) AS total, COUNTIF(district_office IS NULL) AS null_districts
    FROM (
        SELECT *
        FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`
        WHERE 
            start_date < CURRENT_DATE() AND (
                termination_date IS NULL OR (
                    termination_date < CURRENT_DATE() AND termination_date > @lower_bound_date
                )
            )
    ) l,
    UNNEST(
        GENERATE_DATE_ARRAY(
            DATE(start_date),
            DATE(COALESCE(termination_date, CURRENT_DATE())), INTERVAL 1 DAY)
    ) AS supervision_date
    WHERE supervision_date = LAST_DAY(supervision_date, MONTH) AND supervision_date > @lower_bound_date
    GROUP BY supervision_date

)
SELECT *, null_districts/total AS null_district_rate
FROM date_counts
ORDER BY supervision_date

,supervision_date,total,null_districts,null_district_rate
0,2018-01-31,44430,3,0.000068
1,2018-02-28,44429,3,0.000068
2,2018-03-31,44159,3,0.000068
3,2018-04-30,44472,3,0.000067
4,2018-05-31,44550,2,0.000045
5,2018-06-30,44058,3,0.000068
6,2018-07-31,44401,1,0.000023
7,2018-08-31,44144,1,0.000023
8,2018-09-30,43978,1,0.000023
9,2018-10-31,44287,0,0.000000


In [245]:
%%bigquery --params {"state_code": "US_PA", "id_type": "US_PA_PBPP"}
WITH 
overlapping_ips AS (
    SELECT DISTINCT person_external_id, person_id, measurement_date
    FROM `recidiviz-staging.state.state_incarceration_period` ip, UNNEST([
        DATE('2019-12-31'),
        DATE('2020-01-31'),
        DATE('2020-02-29'),
        DATE('2020-03-31'),
        DATE('2020-04-30'),
        DATE('2020-05-31')
    ]) AS measurement_date
    JOIN (
        SELECT state_code, person_id, external_id AS person_external_id
        FROM `recidiviz-staging.state.state_person_external_id` eid
        WHERE eid.id_type = @id_type
    )
    USING (state_code, person_id)
    WHERE state_code = @state_code
        AND admission_date <= measurement_date 
        AND (release_date IS NULL OR release_date > measurement_date)
        AND custodial_authority != 'SUPERVISION_AUTHORITY'
        AND ip.external_id IS NOT NULL
),
pa_reported_people AS (
    SELECT *
    FROM `recidiviz-123.validation_external_accuracy_tables.supervision_population_person_level`
    WHERE region_code = @state_code
),
recidiviz_reported_people AS (
    SELECT l.*, supervision_date
    FROM (
        SELECT *
        FROM `recidiviz-staging.ageiduschek_supervision_period_validation.supervision_period_latest`
        WHERE 
            start_date < CURRENT_DATE() AND (
                termination_date IS NULL OR (
                    termination_date < CURRENT_DATE() AND termination_date > '2019-12-31'
                )
            )
    ) l,
    UNNEST(
        GENERATE_DATE_ARRAY(
            DATE(start_date),
            DATE(COALESCE(termination_date, CURRENT_DATE())), INTERVAL 1 DAY)
    ) AS supervision_date
    LEFT OUTER JOIN overlapping_ips
    ON 
        overlapping_ips.person_external_id = parole_number AND
        overlapping_ips.measurement_date = supervision_date
    WHERE supervision_date = LAST_DAY(supervision_date, MONTH) 
        AND supervision_date >= '2019-12-31' AND supervision_date <= '2020-06-01'
        AND overlapping_ips.person_external_id IS NULL
),
comparison AS (
    SELECT
        COALESCE(pa_reported_people.date_of_supervision, recidiviz_reported_people.supervision_date) AS date_of_supervision,
        pa_reported_people.person_external_id AS external_parole_number,
        pa_reported_people.district AS external_district,
        recidiviz_reported_people.parole_number AS internal_parole_number,
        recidiviz_reported_people.district_office AS internal_district
    FROM 
        pa_reported_people
    FULL OUTER JOIN
        recidiviz_reported_people
    ON 
     pa_reported_people.person_external_id = recidiviz_reported_people.parole_number AND 
     pa_reported_people.date_of_supervision = recidiviz_reported_people.supervision_date
),
counts AS (
    SELECT 
        date_of_supervision AS d,
        COUNT(*) AS total,
        COUNTIF(
            internal_parole_number IS NOT NULL AND 
            external_parole_number IS NOT NULL AND 
            external_parole_number = internal_parole_number AND
            external_district IS NOT NULL AND 
            internal_district IS NOT NULL AND 
            external_district = internal_district
        ) AS correct,
        COUNTIF(external_parole_number IS NULL) AS pa_missing,
        COUNTIF(internal_parole_number IS NULL) AS recidiviz_missing,
        COUNTIF(internal_parole_number IS NOT NULL AND external_parole_number IS NOT NULL) AS correct_ppl,
        COUNTIF(external_parole_number = internal_parole_number AND internal_district IS NULL) AS null_districts,
        COUNTIF(
            internal_parole_number IS NOT NULL AND 
            external_parole_number IS NOT NULL AND 
            external_parole_number = internal_parole_number AND 
            internal_district IS NOT NULL AND external_district != internal_district
        )  AS nonnull_wrong_district
    FROM 
        comparison
    GROUP BY date_of_supervision
)
SELECT 
    *,
    1-(correct/total) AS overall_error,
    (pa_missing + recidiviz_missing)/(pa_missing + recidiviz_missing + correct_ppl) AS population_error,
    (null_districts+nonnull_wrong_district)/correct_ppl AS incorrect_district_error
FROM counts

,d,total,correct,pa_missing,recidiviz_missing,correct_ppl,null_districts,nonnull_wrong_district,overall_error,population_error,incorrect_district_error
0,2020-01-31,42307,40944,510,783,41014,0,70,0.032217,0.030562,0.001707
1,2020-02-29,41911,40641,437,780,40694,0,53,0.030302,0.029038,0.001302
2,2020-03-31,42449,41163,411,707,41331,0,168,0.030295,0.026337,0.004065
3,2020-04-30,42599,41560,272,654,41673,0,113,0.024390,0.021738,0.002712
4,2020-05-31,42693,41580,277,727,41689,0,109,0.026070,0.023517,0.002615
5,2019-12-31,42242,40834,541,827,40874,0,40,0.033332,0.032385,0.000979
